In [1]:
from __future__ import division
import math
import os
from collections import defaultdict
import re
import sys
import random
import itertools
from collections import Counter
from sklearn.preprocessing import normalize
import numpy as np
import itertools
import math
import scipy.misc

In [1113]:
# Allowed segmentations for the sentence

output_vocab = ["O","Name","END"]
Max_seglength=3
trans_prob = {('Name', 'END'): 0.17666666666,
 ('Name', 'Name'): 0.0,
 ('Name', 'O'): 0.3111111111111111,
 ('O', 'Name'): 0.3222222222222222,
 ('O', 'O'): 0.4333333333333333,
 ('O', 'END'): 0.033333,
 ('END', 'O'): 0.0,
 ('END', 'Name'): 0.0,
 ('END', 'END'): 0.0,
             }
trans_prob

{('END', 'END'): 0.0,
 ('END', 'Name'): 0.0,
 ('END', 'O'): 0.0,
 ('Name', 'END'): 0.17666666666,
 ('Name', 'Name'): 0.0,
 ('Name', 'O'): 0.3111111111111111,
 ('O', 'END'): 0.033333,
 ('O', 'Name'): 0.3222222222222222,
 ('O', 'O'): 0.4333333333333333}

In [12]:
def indicator(condition):
    if condition is True:
        return 1
    else:
        return 0
    

In [581]:
words="I went to Blue Heron".split()
words[2:4]

def issegcap(words,seg_start,seg_end):
    count=0
    for word in words[seg_start:seg_end+1]:
        count+=indicator(word.istitle())
    if count== seg_end-seg_start+1:
        return 1
    else:
        return 0

def isseglow(words,seg_start,seg_end):
    count=0
    for word in words[seg_start:seg_end+1]:
        count+=indicator(not word.istitle())
    if count== seg_end-seg_start+1:
        return 1
    else:
        return 0
    
       

1

In [1117]:
def features_per_segment(label,prev_label,word,seg_start,seg_end):
    feature_vector=np.zeros((13,1))
    for j in range(len(feature_vector)):
        
        ### Feature based on longer segments
        
        # Capital, Name , prev not name
        feature_vector[0][0]= (issegcap(word,seg_start,seg_end)*indicator(seg_end!=seg_start)*indicator(label=="Name" and prev_label is not "Name"))
        
        # Capital, Name , prev is also name
        feature_vector[1][0]= (issegcap(word,seg_start,seg_end)*indicator(seg_end!=seg_start)*indicator(label=="Name" and prev_label=="Name"))
        # Capital, O
        feature_vector[2][0]= issegcap(word,seg_start,seg_end)*indicator(seg_end!=seg_start)*indicator(label is not "Name")
        # Lower case, O
        feature_vector[3][0]= isseglow(word,seg_start,seg_end)*indicator(seg_end!=seg_start)*indicator(label is not "Name")
        # Lower case, Name
        feature_vector[4][0]= isseglow(word,seg_start,seg_end)*indicator(seg_end!=seg_start)*indicator(label== "Name")
    
    
        ### Feature based on word lengths if it is labelled Name
        
        feature_vector[5][0]= indicator(seg_end!=seg_start)*sum([len(word[t]) for t in range(seg_start,seg_end+1)])*issegcap(word,seg_start,seg_end)*indicator(label=="Name")
        
        ### Feature based on 1-length segments
        
        # Lowercase, Name
        
        feature_vector[6][0]= indicator(seg_end==seg_start)*indicator(label=="Name")*indicator(word[seg_start].islower())
        # Capital,Name
        feature_vector[7][0]= indicator(seg_end==seg_start)*indicator(label=="Name")*indicator(word[seg_start].istitle())
        # Capital,O
        feature_vector[8][0]= indicator(seg_end==seg_start)*indicator(label is not "Name")*indicator(word[seg_start].istitle())
        
        # Lowercase, O
        feature_vector[9][0]= indicator(seg_end==seg_start)*indicator(label is not "Name")*indicator(word[seg_start].islower())
        
        
        
        # Feature based on lengths of segment
        feature_vector[10][0]= indicator(seg_end!=seg_start)*(((seg_end-seg_start+1)-len(word)/3)**2)
        
        # Feature based on transition probabilities
        feature_vector[11][0]= 100*trans_prob[(prev_label,label)]
        
        # Feature based on lower case words
        feature_vector[12][0]= (-10)*(isseglow(words,seg_start,seg_end))*indicator(seg_end!=seg_start)
        
        

        
    return feature_vector



In [874]:
weights=np.array([[8],[-100],[3],[15],[-60],[70],[9],[-3],[-20],[-30],[-60],[60],[10]])
weights

array([[   8],
       [-100],
       [   3],
       [  15],
       [ -60],
       [  70],
       [   9],
       [  -3],
       [ -20],
       [ -30],
       [ -60],
       [  60],
       [  10]])

In [775]:
def feature_potentials(weights,feature_vector):
# feature_params, C x n numpy array of feature parameter.
# word ,a w x n numpy array of feature vectors,
# word= X_test[i]
    return (np.dot(np.transpose(weights),feature_vector))


In [1118]:
X1= "I went to Blue Heron".split()
s1=[(0,0,"O"),(1,1,"O"),(2,2,"O"),(3,4,"Name")]
s2=[(0,0,"O"),(1,1,"O"),(2,2,"O"),(3,3,"Name"),(4,4,"Name")]
print "Segmentation 1" ,s1 ,"\n"
print "Segmentation 2" ,s2 ,"\n"

featpot1= (feature_potentials(weights,features_per_segment("Name","O",X1,3,4)))
featpot2= (feature_potentials(weights,features_per_segment("Name","O",X1,3,3)))+ (feature_potentials(weights,features_per_segment("Name","Name",X1,4,4)))

print "Dot product for S1 " , featpot1
print "Dot product for S2 ", featpot2

Segmentation 1 [(0, 0, 'O'), (1, 1, 'O'), (2, 2, 'O'), (3, 4, 'Name')] 

Segmentation 2 [(0, 0, 'O'), (1, 1, 'O'), (2, 2, 'O'), (3, 3, 'Name'), (4, 4, 'Name')] 

Dot product for S1  [[ 376.88888889]]
Dot product for S2  [[ 76.66666667]]


In [1121]:
features_per_segment("Name","O",X1,3,4)

array([[  1.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  9.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.11111111],
       [ 32.22222222],
       [  0.        ]])

In [1120]:
features_per_segment("Name","O",X1,3,4)

array([[  1.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  9.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.11111111],
       [ 32.22222222],
       [  0.        ]])

In [1119]:
features_per_segment("Name","END",X1,4,4)

array([[ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.]])

In [79]:
def dict_argmax(dct):
    """Return the key whose value is largest. In other words: argmax_k dct[k].
    Behavior undefined if ties (python docs might give clues)"""
    return max(dct.iterkeys(), key=lambda k: dct[k])

In [1036]:
def viterbidict(X, output_vocab):


    N = len(X)   # length of input sentence
    # viterbi log-prob tables
    V = [{tag:None for tag in output_vocab } for t in range(N)]
    # backpointer tables
    # back[0] could be left empty. it will never be used.
    back = [{tag:None for tag in output_vocab } for t in range(N)]
    # Storing back pointers
    backprop=[{tag:{t:None for t in range(1,N+1)} for tag in output_vocab} for t in range(N)]             
    
    for i in range(N):
        for y in output_vocab:
            viterbi_list=[]
            for ybar in output_vocab:
                for d in range(1,N+1):
                    # Handling i=0 case
                    if i<0:
                        viterbi_list.append(-float("inf"))
                        V[i][y]= max(viterbi_list)
                    # Handling i=0 case
                    if i ==0:
                        viterbi_list.append(0)
                        V[i][y]= max(viterbi_list)
                        #back[t][k]=0 ## Initializing to zero, even though it isn't used.
                    if i-d <0:
                        viterbi_list.append(-float("inf"))
                        V[i-d][y]= max(viterbi_list)
                    else:
                        pot=(feature_potentials(weights,features_per_segment(y,ybar,X,i-d,i))[0][0])
                        viterbi_list.append(V[i-d][ybar]+pot )
                        backprop[i][ybar][d]=(V[i-d][ybar])
                        V[i][y]= max(viterbi_list)

                back[i][y]=dict_argmax(backprop[i][y])
    print V,"\n \n"
    print back,"\n \n"
    print backprop,"\n \n"
                        

    return V,back,backprop

In [1045]:
def viterbi(X, output_vocab):

    N = len(X)   # length of input sentence
    # viterbi trellis        
    V = np.zeros((N+1,len(output_vocab)),dtype=float) 
    # back trace for tag
    BackTag = np.zeros((N+1,len(output_vocab)),dtype=int) 
    # back trace for segment lengths
    SegLen = np.zeros((N+1,len(output_vocab)),dtype=int) 
    for i in range(1,N+1):
        for y in range(len(output_vocab)):
            maxx = -sys.maxint    # or -float("inf")
            argmax_y = 0
            argmax_d = 0
            for ybar in range(len(output_vocab)):
                for d in range(1,4):
                    if i>=d:
                        pot= (feature_potentials(weights,features_per_segment(output_vocab[y],output_vocab[ybar],X,i-d,i-1))[0][0])
                        val = V[i-d,ybar] + pot
                        
                        if val>maxx:
                            maxx =val
                            argmax_d = d
                            argmax_y = ybar
            V[i,y] = maxx
            BackTag[i,y] = argmax_y
            SegLen[i,y] = argmax_d
    return V,BackTag,SegLen
    
    

In [1105]:
def segmentdict(V,back):
    labels=[]
    lengths=[]
    for i in range(len(V)-1,-1,-1):
        if max(V[i].values()) is not 0:
            tag=dict_argmax(V[i])
            seg=back[i][tag]
            labels.append(tag)
            lengths.append(seg)
    labels=labels[::-1]
    lengths=lengths[::-1]
    segments=[]

    for j in range(len(labels)):
        segments.append((j , j+lengths[j]-1,labels[j] ))
    return segments

def segmentation(X,V,BackTag,SegLen):
    N=len(X)
    last_tag  = np.argmax(V[N])
    segmentation = []
    d=N
    while (d>0):
        next_tag=BackTag[d,last_tag]
        dnext=SegLen[d,last_tag]
        print  X[d-dnext:d], " is :",output_vocab[last_tag]
        segmentation.append((BackTag[d,last_tag],SegLen[d,last_tag]))
        last_tag=next_tag
        d=d-dnext

In [1108]:
weights=np.array([[10],[-20],[-14],[-6],[-7],[30],[-20],[-10],[-17],[90],[2],[3],[-4]])
weights

array([[ 10],
       [-20],
       [-14],
       [ -6],
       [ -7],
       [ 30],
       [-20],
       [-10],
       [-17],
       [ 90],
       [  2],
       [  3],
       [ -4]])

In [1106]:

X1="I went To Blue Heron ".split()
X2="I went skiing with Fernando Pereira in British Columbia".split()
X3= "This assignment was Extremely Tough but I managed To get It Working".split()
V1,backtag1,seglen1= viterbi(X1, output_vocab)
segmentation(X1,V1,backtag1,seglen1)
print "\n"
V2,backtag2,seglen2= viterbi(X2, output_vocab)
segmentation(X2,V2,backtag2,seglen2)
print "\n"
V3,backtag3,seglen3= viterbi(X3, output_vocab)
segmentation(X3,V3,backtag3,seglen3)

['Blue', 'Heron']  is : Name
['To']  is : O
['went']  is : O
['I']  is : O


['British', 'Columbia']  is : Name
['in']  is : O
['Fernando', 'Pereira']  is : Name
['with']  is : O
['skiing']  is : O
['went']  is : O
['I']  is : O


['It', 'Working']  is : Name
['get']  is : O
['To']  is : O
['managed']  is : O
['I']  is : O
['but']  is : O
['Extremely', 'Tough']  is : Name
['was']  is : O
['assignment']  is : O
['This']  is : O
